In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    import os
    import sys
    os.chdir('/content/drive/MyDrive/DL_Project_GD/')
    sys.path.append('/content/drive/MyDrive/DL_Project_GD/code/')

    #%% [code]
    ! pip install -qqq "torchvision" "torch>=1.6, <1.9" "torchmetrics>=0.3" "ipython[notebook]" "pytorch-lightning>=1.3" "torchtext"
    ! pip install -qqq cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

    ## Install dependencies
    ! pip install -qqq wget
    ! apt-get -qqq install sox libsndfile1 ffmpeg
    ! pip install -qqq unidecode
    ! pip install -qqq matplotlib>=3.3.2

    ## Install NeMo
    BRANCH = 'r1.6.1'
    ! python -m pip install -qqq git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

    ! mkdir configs
    ! wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

    datapath = './SyncData/raw/'
    rootpath = './'
    print("Running with Colab setup.")
    tpu_cores = 8
    gpu_cores = None
except(ModuleNotFoundError):
    print('Initialising non-colab set-up...')
    tpu_cores=None
    gpu_cores=None
    datapath = '../dataset/processed/'
    rootpath = '../'

from VGGnet import VGGnet
from dataloader import VoxDataloader
import pytorch_lightning.loggers as pl_loggers
from pytorch_lightning.callbacks import EarlyStopping
import pytorch_lightning as pl
print("datapath: {datapath}")
print('done.')

In [ ]:
config = {
            # learning params
            'lr': 1e-3,
            'batch_size': 32,
            'early_stopping': True,
            'optimizer': 'SGD',
            'max_epochs': 100,
            'early_stopping': True,
            'patience': 5, # tolerance for early stopping

            #, regularization
            'dropout': 0.5,
            'L2': 0.1,
            'batch_norm': True,
            'gradient_clipping': 0.5,
}

In [ ]:
logname = "opt{}_lr{}_reg{}_drop{}_bn{}_gc{}".format(*[config[s] for s in ['optimizer','lr','L2','dropout','batch_norm','gradient_clipping']])

dataloader = VoxDataloader(datapath, batch_size=config['batch_size'])
model = VGGnet(num_classes=dataloader.num_classes(), lr=config['lr'], batch_norm=config['batch_norm'])
tb_logger = pl_loggers.TensorBoardLogger(rootpath + './VGGlogs/', name=logname)
trainer = pl.Trainer(logger=tb_logger, max_epochs=config['max_epochs'], tpu_cores=tpu_cores, gpus=gpu_cores, log_every_n_steps=20,
                      callbacks=[EarlyStopping(
                          monitor='val_loss',
                          patience=config['patience'])] if config['early_stopping'] else None)

trainer.fit(model, dataloader)